# main

In [49]:
import json
import requests
import popelines
import os
from main import fix_values, populating_vals, flatten, flatten_dupe_vals

In [2]:
def fix_values(value, key, reset_key):
    if key == reset_key:
        new_list = []
        for x in value:
            value[x][f'{reset_key}_id'] = x
            new_list.append(value[x])
        return new_list
    else:
        return value

In [3]:
# read endpoint, returns a json file of the HTTP request
def read_endpoint(endpoint, headers_set, params_set=None):
    try:
        response = requests.get(endpoint, headers=headers_set, params=params_set)
        response_text = json.loads(response.text)
        response.raise_for_status()

    except requests.exceptions.HTTPError as err:
        print(err)

    return response_text

In [4]:
# generate all experiments given a project id
def generate_experiments(project_id, experiment_endpoint, experiment_headers, experiment_params):
    # get all experiments from one project
    j_exp = read_endpoint(endpoint=experiment_endpoint, headers_set=experiment_headers, params_set=experiment_params)

    # loop for all experiments in this project
    experiment_id_list = []
    upload_exp_list = []
    for exp in j_exp:
        exp['project_id'] = project_id
        upload_exp_list.append(flatten(exp, {}, ''))
        experiment_id_list.append(exp['id'])
        
    return upload_exp_list, experiment_id_list

In [5]:
# generate all projects within account
def generate_projects(project_endpoint, project_headers):
    # get all projects
    j_proj = read_endpoint(endpoint=project_endpoint, headers_set=project_headers)

    # store a list of project metadata
    return j_proj

# load all projects, get project_id's

In [89]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
    'Content-Type': 'application/json',
}

In [77]:
project_endpoint = 'https://api.optimizely.com/v2/projects'

In [78]:
all_projects = generate_projects(project_endpoint, headers)

In [ ]:
# pope.write_to_json(file_name='../uploads/projects.json', jayson=all_projects, mode='w')

In [ ]:
# pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
# print("Successfully uploaded all projects.")

In [79]:
project_id_list = []
for project in all_projects:
    project_id_list.append((project['id']))

In [80]:
project_id_list


[9965963792, 10347872424, 10427612860, 10429921250, 10849384554]

In [81]:
project_id_name = []
for project in all_projects:
    project_id_name.append((project['id'], project['name']))

In [82]:
project_id_name

[(9965963792, 'Onboarding'),
 (10347872424, 'Corporate Website'),
 (10427612860, 'Marketo Pages'),
 (10429921250, 'Corporate Website - Landing Pages'),
 (10849384554, 'Corporate Website - Copy')]

# load experiments

In [83]:
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'


In [84]:
experiment_id_list = []

for project_id in project_id_list:
    # params include project_id (required) and experiments pulling per each request (default only 25)
    params = (
        ('project_id', project_id),
        ('per_page', 100),
    ) 

    exp_list, exp_id_list = generate_experiments(project_id, experiment_endpoint, headers, params)
    experiment_id_list.extend(exp_id_list)
    
    # upload experiments
    pope.write_to_json(file_name='../uploads/experiments.json', jayson=exp_list, mode='w')
    pope.write_to_bq(table_name='experiments', file_name='../uploads/experiments.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
    print(f"Successfully uploaded experiments for project {project_id}")

INFO:root:Writing experiments to BQ from file ../uploads/experiments.json


2019-02-14 16:36:21,365 - root - Writing experiments to BQ from file ../uploads/experiments.json
Successfully uploaded experiments for project 9965963792


INFO:root:Writing experiments to BQ from file ../uploads/experiments.json


2019-02-14 16:36:30,932 - root - Writing experiments to BQ from file ../uploads/experiments.json
Successfully uploaded experiments for project 10347872424


INFO:root:Writing experiments to BQ from file ../uploads/experiments.json


2019-02-14 16:36:35,594 - root - Writing experiments to BQ from file ../uploads/experiments.json
Successfully uploaded experiments for project 10427612860


INFO:root:Writing experiments to BQ from file ../uploads/experiments.json


2019-02-14 16:36:44,673 - root - Writing experiments to BQ from file ../uploads/experiments.json
Successfully uploaded experiments for project 10429921250


INFO:root:Writing experiments to BQ from file ../uploads/experiments.json


2019-02-14 16:36:49,731 - root - Writing experiments to BQ from file ../uploads/experiments.json
Successfully uploaded experiments for project 10849384554


# results timeseries

In [93]:
experiment_id_list

[10756790433,
 10796678057,
 10800885728,
 10815382176,
 10945903729,
 10955300529,
 11039523400,
 11938601024,
 11966791385,
 11992110033,
 10488931082,
 10545851126,
 10556110251,
 10564323775,
 10605041742,
 10622090898,
 10666463868,
 10675031376,
 10681551672,
 10683631399,
 10715090552,
 10733622163,
 10797220881,
 10797550884,
 10805332324,
 10844011372,
 10854082584,
 10882272812,
 11011180459,
 11066932017,
 11085711684,
 11087580674,
 11174107321,
 11189744252,
 11191727326,
 11191905720,
 11199410199,
 11206301430,
 11308083317,
 11334340375,
 11356331084,
 11367191147,
 11378763890,
 11396076650,
 11404885421,
 11406773739,
 11432181239,
 11474010139,
 11482931694,
 11556750451,
 11562826019,
 11570380944,
 11657860282,
 11713452853,
 11720710615,
 11722600722,
 12016386767,
 12096030597,
 12117040288,
 12132980378,
 12166698246,
 12169545706,
 12239002639,
 12240020740,
 12275901710,
 12337280003,
 12850290083,
 12903040293,
 13026380185,
 13026390548,
 13559460138,
 11377

In [97]:
result_endpoint = f'https://api.optimizely.com/v2/experiments/10854082584/timeseries'

In [91]:
response_ts = requests.get(result_endpoint, headers=headers)

# if '' then the experiment has not started yet
if response_ts.text == '':
    j_ts = {'experiment_id': experiment_id}
elif 'bad' in response_ts.text:
    j_ts = {'experiment_id': experiment_id}
else:
    j_ts = json.loads(response_ts.text)

new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')


In [66]:
from datetime import datetime

In [67]:
flattened_metrics = []
for metric in new_j_ts['metrics']:
    for ts in metric['results']:
        flattened_timeseries = []
        for element in ts['timeseries']:
            element['upload_ts'] = str(datetime.now())
            flattened_timeseries.append(flatten(element, {}, ''))

        # Replace old 'timeseries' with new 'flattened_timeseries'
        updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
        flattened_results = flatten_dupe_vals(vals=updated_results, key='timeseries')

    # Replace old 'metrics' with new 'flattened_results'
    update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
    flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))
    

KeyError: 'metrics'

In [68]:
flattened_metrics

[]

In [98]:
response_ts = requests.get(result_endpoint, headers=headers)

In [102]:
'bad' in response_ts.text

True